In [2]:
import tensorflow as tf
tf.__version__

'1.13.1'

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1], padding='SAME')

def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W) + b)

def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

* This specifies the weights for either fully connected or convolutional layers of the
network. They are initialized randomly using a truncated normal distribution
with a standard deviation of .1. This sort of initialization with a random normal
distribution that is truncated at the tails is pretty common and generally pro‐
duces good results (see the upcoming note on random initialization).
* This defines the bias elements in either a fully connected or a convolutional layer.
These are all initialized with the constant value of .1 .
* This specifies the convolution we will typically use. A full convolution (no skips)
with an output the same size as the input.
* This sets the max pool to half the size across the height/width dimensions, and in
total a quarter the size of the feature map.
* This is the actual layer we will use. Linear convolution as defined in conv2d , with
a bias, followed by the ReLU nonlinearity.
* A standard full layer with a bias. Notice that here we didn’t add the ReLU. This
allows us to use the same layer for the final output, where we don’t need the non‐
linear part.

In [8]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])
conv1 = conv_layer(x_image, shape=[5, 5, 1, 32])
conv1_pool = max_pool_2x2(conv1)

conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

conv2_flat = tf.reshape(conv2_pool, [-1, 7 * 7 * 64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full1_drop, 10)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
